In [104]:
info = os.listdir('/Users/alexbraun/Downloads/data/arroway_wood/data')
info = filter(lambda x: re.search('diffuse', x), info)
info = filter(lambda x: re.search('_a_', x), info)
info = map(lambda x: re.search('.*_(.*)_diffuse', x).group(1), info)
info = sorted(info)
idwood = pd.read_csv('/Users/alexbraun/google_drive/code/projects/texture_classifier/data/identifying_wood_data.csv')

temp = list(set(list(chain(*map(lambda x: x.split('-'), info)))))
def has_keyword(item):
    output = []
    for regex in temp:
        found = re.search(regex, item, re.IGNORECASE)
        if found:
            output.append(found.group(0))
    if output:
        return ' '.join(output)
    return False
    
for col in ['common_name']:#, 'scientific_name']:
    new_col = 'found_' + col
    idwood[new_col] = None
    idwood[new_col] = idwood[col].apply(has_keyword)
    
x = idwood[(idwood.found_common_name != False)]# | (idwood.found_scientific_name != False)]

In [101]:
sci = x[x.found_scientific_name != False].index
# x.found_scientific_name.tolist()

In [115]:
mask = x.found_common_name.apply(lambda x: x not in ['fir', 'cedar', 'white', 'black', 'red'])
x = x[mask][['common_name', 'found_common_name']]

In [11]:
# %%snakeviz
%cd /Users/alexbraun/google_drive/code/projects/texture_classifier/python 
from sklearn.ensemble import RandomForestClassifier

import core.image_scanner
reload(core.image_scanner)
from core.image_scanner import ImageScanner

import core.utils
reload(core.utils)
from core.utils import *

# root = '/Volumes/abraun.backup.01/project folders/texture_classifier/data'
root = '/Users/alexbraun/Documents/projects/texture_classifier/data'

spec = ['texture', 'image_id', 'image_class', 'common_name', 'pass_', 'extension']
info = get_series_info(root, spec)
info = info[(info.image_class == 'a') & (info.pass_ == 'diffuse')]
info = info.head()

patches = 1000

y = info.common_name.apply(lambda x: [x]) * patches
y = list(chain(*y.tolist()))

params = {
            'aspect_ratio':     1,
            'min_size':         0.05,
            'max_size':         0.1,
            'patches':          patches,
            'patch_resolution': (100, 100),
            'rotation':         'random'
}

# %time data = get_data(info, params)

/Users/alexbraun/google_drive/code/projects/texture_classifier/python


In [194]:
# X.tofile('/Users/alexbraun/Documents/projects/texture_classifier/data/5-sample_1000-per-sample_100x100_rotation-dependent.txt')

In [221]:
clf = RandomForestClassifier()
params = {
    'n_estimators':        [10],
#     'criterion':'          gini',
#     'max_depth':           None,
#     'min_samples_split':   2,
#     'min_samples_leaf':    1,
#     'max_features':'       'auto',
#     'max_leaf_nodes':      None,
#     'bootstrap':           True,
#     'oob_score':           False,
    'n_jobs':              [-1]
#     'random_state':        None,
#     'verbose':             0,
#     'min_density':         None,
#     'compute_importances': None
}

train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.2)
grid = GridSearchCV(clf, params)
grid.fit(train_x, train_y)
print(grid.best_score_)
print(grid.best_estimator_.score(test_x, test_y))

GridSearchCV(cv=None,
       estimator=RandomForestClassifier(bootstrap=True, compute_importances=None,
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_density=None, min_samples_leaf=1,
            min_samples_split=2, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0),
       fit_params={}, iid=True, loss_func=None, n_jobs=1,
       param_grid={'n_estimators': [10], 'n_jobs': [-1]},
       pre_dispatch='2*n_jobs', refit=True, score_func=None, scoring=None,
       verbose=0)